In [1]:
from __future__ import print_function, division
from data_utils import load_CIFAR10

X_train, y_train, X_test, y_test = load_CIFAR10("cifar-10-batches-py")

print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

X_train: (50000, 32, 32, 3)
y_train: (50000,)
X_test: (10000, 32, 32, 3)
y_test: (10000,)


In [19]:
from skimage.feature import local_binary_pattern
from skimage import color
import numpy as np

radius = 3
n_points = 8 * radius
METHOD = 'uniform'
eps = 1e-7

lbp_train = []
for i in xrange(X_train.shape[0]):
    gray_img = color.rgb2gray(X_train[i])
    lbp = local_binary_pattern(gray_img, n_points, radius, METHOD)
    h, _ = np.histogram(lbp.ravel(),bins=np.arange(0, n_points+3), range=(0, n_points+2),normed=True)
    h = h[np.newaxis, :]
    lbp_train.append(h)
    
lbp_train = np.concatenate(lbp_train, axis=0)
print(lbp_train.shape)

lbp_test = []
for i in xrange(X_test.shape[0]):
    gray_img = color.rgb2gray(X_test[i])
    lbp = local_binary_pattern(gray_img, n_points, radius, METHOD)
    h, _ = np.histogram(lbp.ravel(),bins=np.arange(0, n_points+3), range=(0, n_points+2),normed=True)
    h = h[np.newaxis, :]
    lbp_test.append(h)

lbp_test = np.concatenate(lbp_test, axis=0)
print(lbp_test.shape)  

(50000, 42)
(10000, 42)


In [9]:
from sklearn import svm
import time

tic = time.time()
lin_clf = svm.LinearSVC() 

lin_clf.fit(lbp_train, y_train)
toc_1 = time.time()
print("time for training: %d seconds" %(toc_1-tic))

preds = lin_clf.predict(lbp_test)
toc_2 = time.time()
print("time for prediction: %d seconds" %(toc_2-toc_1))

acc = (preds==y_test).mean()
print("accuracy: ", acc)

time for training: 9 seconds
time for prediction: 0 seconds
accuracy:  0.2418


In [18]:
from sklearn.ensemble import RandomForestClassifier
import time
tic = time.time()
rf = RandomForestClassifier(max_depth = 20, n_jobs = -1)
rf.fit(lbp_train,y_train)
toc_1 = time.time()
print("time for training: %d seconds" %(toc_1-tic))

preds = rf.predict(lbp_test)
toc_2 = time.time()
print("time for prediction: %d seconds" %(toc_2-toc_1))

acc = (preds==y_test).mean()
print("test accuracy: ", acc)

preds = rf.predict(lbp_train)
acc = (preds==y_train).mean()
print("train accuracy: ", acc)

time for training: 0 seconds
time for prediction: 0 seconds
test accuracy:  0.2244
train accuracy:  0.97454
